Source: https://colab.research.google.com/github/DhavalTaunk08/Transformers_scripts/blob/master/Transformers_multilabel_distilbert.ipynb#scrollTo=StbPlIyKDP9E

In [ ]:
import pandas as pd
pd.read_parquet("hf://datasets/data-is-better-together/fineweb-c/dan_Latn/train-00000-of-00001.parquet")

<a id='section01'></a>
### Importing Python Libraries and preparing the environment

At this step we will be importing the libraries and modules needed to run our script. Libraries are:
* Pandas
* Pytorch
* Pytorch Utils for Dataset and Dataloader
* Transformers
* BERT Model and Tokenizer

Followed by that we will preapre the device for GPU execeution. This configuration is needed if you want to leverage on onboard GPU.

*I have included the code for TPU configuration, but commented it out. If you plan to use the TPU, please comment the GPU execution codes and uncomment the TPU ones to install the packages and define the device.*

In [29]:
# Installing the transformers library and additional libraries if looking process

!pip install -q transformers

# Code for TPU packages install
# !curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

In [ ]:
# Importing stock ML Libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from collections import Counter
import ast
import matplotlib.pyplot as plt

In [31]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

<a id='section02'></a>
### Importing and Pre-Processing the domain data

We will be working with the data and preparing for fine tuning purposes.
*Assuming that the `train.csv` is already downloaded, unzipped and saved in your `data` folder*

* Import the file in a dataframe and give it the headers as per the documentation.
* Taking the values of all the categories and coverting it into a list.
* The list is appened as a new column and other columns are removed

In [32]:
def Most_common_label(label_list):
    label = Counter(label_list).most_common(1)[0][0]
    inx = sort_order[label]  # Most frequent label
    L = [0 for i in range(len(unique_labels))]
    L[inx] = 1
    return L


def Soft_label(label_list):
    numeric_annotations = [sort_order[label] for label in label_list]
    return np.bincount(numeric_annotations, minlength=len(unique_labels)) / len(label_list)

In [33]:
#########
DATASET = pd.read_parquet("hf://datasets/data-is-better-together/fineweb-c/dan_Latn/train-00000-of-00001.parquet")
PROBLEMATIC_CONTENT = False
LABEL_FUNCTION = Most_common_label
#########


df = pd.DataFrame()
df["text"] = DATASET["text"]
df["educational_value_labels"] = DATASET["educational_value_labels"]
df["problematic_content_label_present"] = DATASET["problematic_content_label_present"]


# REMOVE PROBLEMATIC LABELS FROM DATASET
df = df[df['problematic_content_label_present'] == PROBLEMATIC_CONTENT]

unique_labels = df["educational_value_labels"].explode().unique().tolist()
sort_order = {
    "None": unique_labels.index("None"),
    "Minimal": unique_labels.index("Minimal"),
    "Basic": unique_labels.index("Basic"),
    "Good": unique_labels.index("Good"),
    "Excellent": unique_labels.index("Excellent"),
}

# Process Data labels
df["Final_label"] = df["educational_value_labels"].apply(LABEL_FUNCTION)

# Display sample rows
df.sample(5)

,text,educational_value_labels,problematic_content_label_present,Final_label
672,Cerfidan ApS er ISO-akkrediteret af DANAK til ...,"[None, None, None]",False,"[0, 1, 0, 0, 0]"
662,Jeg kan varmt anbefale E-forsikringer. Jeg har...,"[None, None]",False,"[0, 1, 0, 0, 0]"
425,"Leder du efter gode kalendergaver til én, du h...","[None, None, None]",False,"[0, 1, 0, 0, 0]"
161,"Skønne fordele\nI C&S Universe, får du en rækk...","[None, None]",False,"[0, 1, 0, 0, 0]"
633,Allergica Bronci D6 - 10 x 1 ml\nProduktbeskri...,"[None, Minimal]",False,"[0, 1, 0, 0, 0]"


In [34]:
new_df = pd.DataFrame()
new_df["text"] = df["text"]
new_df["labels"] = df["Final_label"]
unique_labels

['Minimal', 'None', 'Basic', 'Excellent', 'Good']

<a id='section03'></a>
### Preparing the Dataset and Dataloader

We will start with defining few key variables that will be used later during the training/fine tuning stage.
Followed by creation of CustomDataset class - This defines how the text is pre-processed before sending it to the neural network. We will also define the Dataloader that will feed  the data in batches to the neural network for suitable training and processing.
Dataset and Dataloader are constructs of the PyTorch library for defining and controlling the data pre-processing and its passage to neural network. For further reading into Dataset and Dataloader read the [docs at PyTorch](https://pytorch.org/docs/stable/data.html)

#### *CustomDataset* Dataset Class
- This class is defined to accept the `tokenizer`, `dataframe` and `max_length` as input and generate tokenized output and tags that is used by the BERT model for training.
- We are using the BERT tokenizer to tokenize the data in the `comment_text` column of the dataframe.
- The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs, namely: `ids`, `attention_mask`, `token_type_ids`
---
- *This is the first difference between the distilbert and bert, where the tokenizer generates the token_type_ids in case of Bert*
---
- To read further into the tokenizer, [refer to this document](https://huggingface.co/transformers/model_doc/bert.html#berttokenizer)
- `targest` is the list of categories labled as `0` or `1` in the dataframe.
- The *CustomDataset* class is used to create 2 datasets, for training and for validation.
- *Training Dataset* is used to fine tune the model: **80% of the original data**
- *Validation Dataset* is used to evaluate the performance of the model. The model has not seen this data during training.

#### Dataloader
- Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of dataloaded to the memory and then passed to the neural network needs to be controlled.
- This control is achieved using the parameters such as `batch_size` and `max_len`.
- Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [ ]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_SIZE = 0.8
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
EPOCHS = 100
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [36]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [37]:
# Creating the dataset and dataloader for the neural network

train_size = TRAIN_SIZE
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (806, 2)
TRAIN Dataset: (645, 2)
TEST Dataset: (161, 2)


In [38]:
train_data.sample(5)

,text,labels
334,Roger Federer slog verdens nummer to Novak Djo...,"[1, 0, 0, 0, 0]"
478,Selv om Louisa May Alcott er ofte forbundet me...,"[1, 0, 0, 0, 0]"
528,Der er et stort projekt under opsejling hos De...,"[1, 0, 0, 0, 0]"
88,Hydrogen er verdens mindste og\nmest effektive...,"[1, 0, 0, 0, 0]"
101,En kæmpe hånd udspillede sig for kort tid side...,"[0, 1, 0, 0, 0]"


In [39]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

<a id='section04'></a>
### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `BERTClass`.
 - This network will have the `Bert` model.  Follwed by a `Droput` and `Linear Layer`. They are added for the purpose of **Regulariaztion** and **Classification** respectively.
 - In the forward loop, there are 2 output from the `BertModel` layer.
 - The second output `output_1` or called the `pooled output` is passed to the `Drop Out layer` and the subsequent output is given to the `Linear layer`.
 - Keep note the number of dimensions for `Linear Layer` is **6** because that is the total number of categories in which we are looking to classify our model.
 - The data will be fed to the `BertClass` as defined in the dataset.
 - Final layer outputs is what will be used to calcuate the loss and to determine the accuracy of models prediction.
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference.

#### Loss Function and Optimizer
 - The Loss is defined in the next cell as `loss_fn`.
 - As defined above, the loss function used will be a combination of Binary Cross Entropy which is implemented as [BCELogits Loss](https://pytorch.org/docs/stable/nn.html#bcewithlogitsloss) in PyTorch
 - `Optimizer` is defined in the next cell.
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

#### Further Reading
- You can refer to my [Pytorch Tutorials](https://github.com/abhimishra91/pytorch-tutorials) to get an intuition of Loss Function and Optimizer.
- [Pytorch Documentation for Loss Function](https://pytorch.org/docs/stable/nn.html#loss-functions)
- [Pytorch Documentation for Optimizer](https://pytorch.org/docs/stable/optim.html)
- Refer to the links provided on the top of the notebook to read more about `BertModel`.

In [40]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        #Change the secound val to the number of classes !!!!!!!!
        self.l3 = torch.nn.Linear(768, len(unique_labels))

    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [84]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

def accuracyTest(outputs, targets):
    outputs = outputs.to(device, dtype=torch.int)
    targets = targets.to(device, dtype=torch.int)

    correct = torch.all(outputs == targets, dim=1)

    correct_num = correct.sum().item()

    accuracy = correct_num / len(targets)

    return accuracy

In [42]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
train_accuracies_pr_epoch = []
val_accuracies_pr_epoch = []
train_losses = []

def train(epoch):
    model.train()
    total_loss = 0  # Track total loss for the epoch
    train_accuracies = []

    for batch_idx,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)
        probs = torch.softmax(outputs, dim=1) # SOFTMAX (dim=1) or SIGMOID (), depends on how we interpret the multiclass
    
        max_indices = torch.argmax(probs, dim=1)
        preds = torch.zeros_like(probs)
        preds.scatter_(1, max_indices.unsqueeze(1), 1)

        print(preds)
        print(targets)
        batch_train_accuracy = accuracyTest(preds, targets)
        train_accuracies.append(batch_train_accuracy)

        print(batch_train_accuracy)
        
        loss = loss_fn(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Print loss every 10 batches (adjust as needed)
        if batch_idx % 10 == 0:
            print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item()}')

    
    train_accuracies_pr_epoch.append(np.mean(train_accuracies)) # Save avrage accuracies for the epoch
    avg_train_loss = total_loss / len(training_loader)
    train_losses.append(avg_train_loss)  # Save average loss for the epoch
    print(f'Epoch {epoch} Average Training Loss: {avg_train_loss}, Avrage Training Accuraccy: {np.mean(train_accuracies)}')

val_accuracies = []
val_f1_micro = []
val_f1_macro = []
test_accuracies_pr_epoch = []

def validation(epoch):
    model.eval()
    fin_targets = []
    fin_outputs = []
    test_acc = []
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.float)
            outputs = model(ids, mask, token_type_ids)

            probs = torch.softmax(outputs, dim=1) # SOFTMAX (dim=1) or SIGMOID (), depends on how we interpret the multiclass
        
            max_indices = torch.argmax(probs, dim=1)
            preds = torch.zeros_like(probs)
            preds.scatter_(1, max_indices.unsqueeze(1), 1)

            # print(preds)
            # print(targets)
            batch_test_acc = accuracyTest(preds, targets)
            test_acc.append(batch_test_acc)
    
    # Calculate metrics
    test_accuracies_pr_epoch.append(np.mean(test_acc)) # Save avrage accuracies for the epoch
    print(f'Epoch {epoch}, Avrage Test Accuraccy: {np.mean(test_acc)}')



In [ ]:
### Epoch Loop

for epoch in range(EPOCHS):
    train(epoch)
    validation(epoch)


In [ ]:
### Plots
epochs = list(range(EPOCHS))

plt.plot(epochs, train_accuracies_pr_epoch, label='Train Accuracy')
plt.plot(epochs, test_accuracies_pr_epoch, label='Validation Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Train vs Validation Accuracy")
plt.legend()
plt.grid(True)

plt.savefig("learningcurve.png")

plt.show()